In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
from typing import Annotated, TypedDict, List, Dict, Sequence
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    outline: Dict[str, str]
    current_section: int
    section_content: str
    section_image: str
    image_prompt: str
    total_sections: int
    full_report: List[Dict[str, str]]


In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search = TavilySearchResults(max_results=3)

개요작성 에이전트

In [6]:
from pydantic import BaseModel, Field, create_model

# 사용자로부터 입력받은 전체 섹션 수에 따라 Data model의 key와 value를 정의하기 위한 함수
# create_model 함수는 Pydantic Data model의 이름과 key:value 값을 기반으로 동적인 Data model을 만둘 수 있게 보조
def create_outline_model(section_count: int):
    fields = {f"section{i}":(str, Field(description=f"Title for section {i}")) for i in range(1, section_count + 1)}
    return create_model("DynamicOutline", **fields)

In [20]:
def outline_generator(state: State):
    DynamicOutline = create_outline_model(state["total_sections"])
    outline_parser = JsonOutputParser(pydantic_object=DynamicOutline)

    outline_prompt = PromptTemplate(
        template="""
        Crate an outlne for a detailed report with exactly {section_count} main sections.
        {format_instructions}
        The topic is: {topic}
        """,
        input_variables=["section_count", "topic"],
        partial_variables={"format_instructions": outline_parser.get_format_instructions()},
    )
    chain = outline_prompt | llm | outline_parser

    outline = chain.invoke({
        "section_count": state["total_sections"],
        "topic": state["messages"][-1].content
    })
    return {"outline": outline}

이미지 생성 에이전트

In [43]:
from openai import OpenAI
from langchain_core.tools import tool

client = OpenAI()

def generate_image(prompt):
    """Generate an image using DALL-E based on the given prompt"""
    response = client.images.generate(
        model="dall-e-3",
        prompt = prompt,
        size="1024x1024",
        quality="standard",
        n=1
    )
    return response.data[0].url

def image_generator(state: State):
    prompt_template = PromptTemplate(
        template="""
        Base on the follwing section cotent, create a prompt for generationg an infographic that represents this section.

        Section content:
        {section_content}

        Image generation prompt(under 1000 characters):
        """,
        input_variables=["section_content"],
    )
    image_prompt = llm.invoke(prompt_template.format(section_content=state["section_content"]))
    image_url = generate_image(image_prompt.content)

    current_section_number = state['current_section']
    current_section = {
        "title": state['outline'][f"section{current_section_number}"],
        "content": state['section_content'],
        "image_url": image_url,
        "image_prompt": image_prompt.content if isinstance(image_prompt, AIMessage) else image_prompt
    }

    updated_full_report = state.get("full_report", []) + [current_section]
    print(f"Completed section {current_section_number} of {state['total_sections']}")

    return {
        "image_prompt": current_section["image_prompt"],
        "section_image": image_url,
        "current_section": current_section_number + 1,
        "full_report": updated_full_report,
    }

LLM설정

In [11]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

컨텐츠 작성 에이전트

In [53]:
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

def contents_writer(state: State):
    if 'error' in state:
        return {'messages': [AIMessage(content=f"An error occured: {state['error']}")]}
    
    if state['current_section'] > state['total_sections']:
        return {'messages': [AIMessage(content=f"Report Completed")]}
    
    current_section_number = state['current_section']
    current_section_key = f"section{current_section_number}"
    current_topic = state['outline'][current_section_key]
    search_results = web_search.invoke(current_topic)

    previous_sections_content = []
    if (current_section_number > 1) :
        for i, section in enumerate(state['full_report']):
            print(f"section_content: {section['title']}\n")
            
            previous_sections_content.append(f"""
            Section {i+1}:
            {section['title']}
            {section['content']}
            """)
    # State에는 죄종만 section_content만 담는다. 확인 필요
    #for i in range(1, current_section_number):
        #section_key = f"section{i}"
        #if section_key in state['section_content']:
            # print(f"section_content: {state['section_content'][section_key]}\n")
            # previous_sections_content.append(f"""
            # Section {i}:
            # {state['outline'][section_key]}
            # {state['section_content'][section_key]}
            # """)
    previous_sections = "\n\n".join(previous_sections_content)

    section_prompt = PromptTemplate(
        template="""
        Write a detailed section for the topic: {topic}.

        Use the following search results from information: {search_results}

        Previous sections:
        {previous_sections}
        Write only the content for this section,
        do not include any image prompts or suggestions.
        Detailed statistics or information is needed, 
        so you should include collected information from search results.
        """,
        input_variables=['topic', 'search_results', 'previous_sections'],
    )

    section_content = llm.invoke(section_prompt.format(
        topic = current_topic,
        search_results = search_results,
        previous_sections = previous_sections
    ))

    return {
        'section_content': section_content.content,
        'current_section': state['current_section'],
    }
            

워드 생성 에이전트

In [59]:
from docx import Document
from docx.shared import Inches
import requests
from io import BytesIO

def report_generator(state: State):
    doc = Document()
    doc.add_heading(f"Report: {state['messages'][0].content}", 0)

    for section in state['full_report']:
        doc.add_heading(section['title'], level=1)
        doc.add_paragraph(section['content'])

        if section['image_url'] != "Image generation failed":
            try:
                response = requests.get(section['image_url'])
                image = BytesIO(response.content)
                doc.add_picture(image, width=Inches(6))
                doc.add_paragraph(f"Image prompt: {section['image_prompt']}")
            except Exception as e:
                doc.add_paragraph(f"Failed to add an image: {str(e)}")

        doc.add_page_break()
    
    filename = f"report {state['messages'][0].content}.docx".replace(" ", " ")
    doc.save(filename)

    return {
        "messages": [AIMessage(content=f"Report is finalized and saved as {filename}")],
        "report_file": filename
    }

In [60]:
graph_builder = StateGraph(State)

graph_builder.add_node("outline_generator", outline_generator)
graph_builder.add_node("contents_writer", contents_writer)
graph_builder.add_node("image_generator", image_generator)
graph_builder.add_node("report_generator", report_generator)

graph_builder.add_edge(START, "outline_generator")
graph_builder.add_edge("outline_generator", "contents_writer")
graph_builder.add_edge("contents_writer", "image_generator")
graph_builder.add_edge("report_generator", END)

def should_continue_writing(state: State):
    if state['current_section'] <= state['total_sections']:
        return "write_section"
    else:
        return "finalize_report"

graph_builder.add_conditional_edges(
    "image_generator",
    should_continue_writing,
    {
        "write_section": "contents_writer",
        "finalize_report": "report_generator",
    },
)

graph = graph_builder.compile()

In [ ]:
from util import display_graph
display_graph(graph)

In [28]:
import os
current_directory = os.getcwd()
print(f"현재 작업 디렉토리: {current_directory}")

현재 작업 디렉토리: /home/user/langchain-kr/98-Agent


In [61]:
from langchain_core.messages import HumanMessage

topic = input("보고서 주제를 입력하세요")
total_sections = int(input("생성할 섹션의 수를 입력하세요: "))

initial_state = {
    "messages": [HumanMessage(content=topic)],
    "total_sections": total_sections,
    "current_section": 1,
}

for chunk in graph.stream(initial_state, stream_mode="update"):
    print(chunk)

print("\n===보고서 생성 완료===")

Completed section 1 of 5
section_content: Introduction to GPTs in the KSD

Completed section 2 of 5
section_content: Introduction to GPTs in the KSD

section_content: Current Landscape of Securities Depositories in Korea

Completed section 3 of 5
section_content: Introduction to GPTs in the KSD

section_content: Current Landscape of Securities Depositories in Korea

section_content: Benefits of Implementing GPTs at KSD

Completed section 4 of 5
section_content: Introduction to GPTs in the KSD

section_content: Current Landscape of Securities Depositories in Korea

section_content: Benefits of Implementing GPTs at KSD

section_content: Challenges and Considerations for GPTs Adoption

Completed section 5 of 5

===보고서 생성 완료===


KSD(Korea Securities Depositories)에서의 GPTs 도입 전략